In [1]:
import os
import yaml
import logging
import numpy as np
import importlib
import subprocess
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [6]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.utils import ml_utils, eda_utils
from src.data import downloader, loader, preprocessor, saving
from src.features import engineer
from src.models import modeling

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = loader.load_config(config_path)

## 3.1. Загрузка данных

### Скачивание данных с kaggle и распаковка архива .zip

In [9]:
downloader.download_and_extract_competition(config)

[⧗] Загружаю данные соревнования skillbox-ml-junior-regression-10 в D:\Skills\Kaggle\ml-regression_concrete-strength\data\raw...
[⧗] Распаковываю skillbox-ml-junior-regression-10.zip...
[✓] Распаковано: skillbox-ml-junior-regression-10.zip
[i] Удален архив: skillbox-ml-junior-regression-10.zip
[✓] Все готово! Данные скачаны и распакованы.


### Загрузка train и test

In [ ]:
# Загрузка train
df_train = loader.data_load(data_type='train', config=config)

In [ ]:
# Загрузка test
df_test = loader.data_load(data_type='test', config=config)

In [ ]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

In [ ]:
# Вывод первых 5 строк тестового датасета
df_test.head()

## 3.2. Предварительная очистка и создание инженерных признаков

In [ ]:
# Удаление неинформативных столбцов и дубликатов
df_train_cleaned = preprocessor.preliminary_data_cleaning(df_train)
df_test_cleaned = preprocessor.preliminary_data_cleaning(df_test)

In [ ]:
# Создание признаков (w_c, SP_pct)
df_train_feat = preprocessor.add_concrete_ratios(df_train_cleaned)
df_test_feat = preprocessor.add_concrete_ratios(df_test_cleaned)

df_train_feat.head()

## 3.3. Оценка объема данных и выбор алгоритмов

In [ ]:
# Формируем список алгоритмов исходя из объема данных по правилу NEPV
models = modeling.check_models_by_nepv(df_train_feat, config)

## 3.4. Проверка порядка признаков в тренировочном и тестовом датасете

In [ ]:
# Создание массива из признаков и массива из целевой переменной
X = df_train_feat.drop(columns=["Strength"])
y = df_train_feat["Strength"]

In [ ]:
# Сравниваем порядок признаков в тренировочном и тестовом датасете
if list(X.columns) == list(df_test_feat.columns):
    print("Порядок признаков совпадает")
else:
    print("Порядок признаков отличается")

## 3.5. Генерация EDA отчета

#### Предварительный отчет

In [ ]:
# Общая информация
df_train_feat.info()

In [ ]:
# Проверка на пропуски train
df_train_feat.isna().sum()

In [ ]:
# Проверка на пропуски test
df_train_feat.isna().sum()

In [ ]:
# Основные статистики train
df_train_feat.describe()

#### Генерация подробного отчета ydataprofaling

In [ ]:
# Генерация отчета по тренировочным данным с помощью ydata-profiling
ml_utils.eda_report(df_train_feat, "train", config)

## 3.6. Анализ выбросов

#### Анализ выбросов

In [ ]:
# Построение сводной таблицы по выбросам на оснвове IQR и значений ГОСТ
summary_df,  outlier_masks_df = preprocessor.detect_outliers(df_train_feat, config)
summary_df

In [ ]:
# Построение графиков 
ml_utils.plot_outliers(df_train_feat, summary_df, max_plots=10)

In [ ]:
# Сохранение отчетов
output_dir=Path().resolve().parent / config["output"]["eda_report_dir"]

saving.save_outliers_report(summary_df, output_dir=output_dir)

## 3.7. Анализ нулей в признаках

#### Анализ признаков с нулнвыми значениями

In [ ]:
config = preprocessor.analyze_zeros(df_train_feat, config)

## 3.8. Анализ зависимости целевой переменнной от признаков

### Анализ корреляции целевой переменнной и признаков

In [ ]:
# Создает комплексный датафрейм c результатами анализа
# корреляции признаков и целевой переменной
df_corr_target = engineer.create_feature_analysis(X, y)
df_corr_target

**Выводы:** Признак Fly Ash показывает очень слабую и статистически незначимую связь с целевой переменной Strength. Поэтому его стоит объединить с другими признаками или удалить

In [ ]:
# Визуализация анализа признаков
ml_utils.visualize_feature_analysis(df_corr_target) 

In [ ]:
# Визуализируем тренды зависимостей таргета от признаков
df_trend_results = ml_utils.plot_feature_trends(df_train_feat,
                                                config, 
                                                target='Strength',
                                                figsize=(16, 50),
                                                alpha=0.25)

In [ ]:
# Выводим датасет преобразований признаков и значений R2
df_trend_results

### Анализ мультиколлинеарности признаков

In [ ]:
# Вывод матрицы корреляций
vif_results = engineer.check_multicollinearity(df=X,
                                               target_column=y,
                                               threshold=5)

In [ ]:
# Отображаем значения VIF для признаков
styled_vif = engineer.highlight_high_vif(vif_results)
display(styled_vif)

**Выводы:** высокая корреляция между признаками Superplasticizer и Sp/C_pct; а также W/C и Cement. Основной вариант действий: удаление признаков Cement и Superplasticizer

## 3.9. Сохранение данных

In [ ]:
# Сохраняем данные в различных форматах, а пути к файлам в config
config = saving.save_cleaned_data(df_train_feat, config)

In [ ]:
# Обновляем congig.yaml
with open(config_path, "w", encoding="utf-8") as f:
    yaml.dump(config, f)